In [ ]:
import pandas as pd

#debug
df_full_raw = pd.read_csv('../data/raw/campeonato-brasileiro-full.csv')
df_stats_raw = pd.read_csv('../data/raw/campeonato-brasileiro-estatisticas-full.csv')

# Definindo os caminhos dos arquivos (usando caminhos relativos da raiz do projeto)
path_full = '../data/raw/campeonato-brasileiro-full.csv'
path_stats = '../data/raw/campeonato-brasileiro-estatisticas-full.csv'
# path_classificacao = '../data/raw/campeonato-brasileiro-classificacao.csv' # carregar se precisar

# Carregando os dataframes
df_full = pd.read_csv(path_full)
df_stats = pd.read_csv(path_stats)

print("Dados carregados com sucesso!")
df_full.head()

Dados carregados com sucesso!


,ID,rodata,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado
0,1,1,29/03/2003,16:00,Guarani,Vasco,NaN,NaN,NaN,NaN,Guarani,Brinco de Ouro,4,2,SP,RJ
1,2,1,29/03/2003,16:00,Athletico-PR,Gremio,NaN,NaN,NaN,NaN,Athletico-PR,Arena da Baixada,2,0,PR,RS
2,3,1,30/03/2003,16:00,Flamengo,Coritiba,NaN,NaN,NaN,NaN,-,Maracanã,1,1,RJ,PR
3,4,1,30/03/2003,16:00,Goias,Paysandu,NaN,NaN,NaN,NaN,-,Serra Dourada,2,2,GO,PA
4,5,1,30/03/2003,16:00,Internacional,Ponte Preta,NaN,NaN,NaN,NaN,-,Beira Rio,1,1,RS,SP


In [2]:
df_full['data'] = pd.to_datetime(df_full['data'],dayfirst=True)
df_full.sort_values(by='data', inplace=True)

df_full.rename(columns={
    'mandante_Placar': 'HomeGoals',
    'visitante_Placar': 'AwayGoals',
    'mandante': 'HomeTeam',
    'visitante': 'AwayTeam'
}, inplace=True)

# Exibir as primeiras linhas e colunas disponíveis
print("Amostra dos Dados:")
print(df_full.head())
print("\nColunas disponíveis para engenharia de features:")
print(df_full.columns)


In [ ]:
# Função para determinar o resultado e os pontos
def get_result_and_points(row):
    if row['HomeGoals'] > row['AwayGoals']:
        return 'H', 3, 0  # Vitória do time da casa
    elif row['HomeGoals'] < row['AwayGoals']:
        return 'A', 0, 3  # Vitória do time visitante
    else:
        return 'D', 1, 1  # Empate

# 1. Atribuir o resultado a NOVAS COLUNAS
df_full[['Result', 'HomePoints', 'AwayPoints']] = df_full.apply(get_result_and_points, axis=1, result_type='expand')

# 2. Codificar a variável alvo (target) para o modelo de ML
# Usaremos 1 para vitória do mandante, 0 para empate e -1 para vitória do visitante
result_mapping = {'H': 1, 'D': 0, 'A': -1}
df_full['Target'] = df_full['Result'].map(result_mapping)

print("DataFrame processado com sucesso! Colunas 'Result', 'HomePoints', 'AwayPoints' e 'Target' foram criadas.")
print(df_full[['HomeTeam', 'AwayTeam', 'HomeGoals', 'AwayGoals', 'Result', 'HomePoints', 'AwayPoints', 'Target']].head())

df_partida_info = df_full[['ID', 'HomeTeam', 'AwayTeam']]

# Adicione esta linha para inspecionar os nomes das colunas de df_stats
print("Colunas em df_stats:", df_stats.columns)

# Você também pode verificar as colunas do outro dataframe para ter certeza
print("Colunas em df_partida_info:", df_partida_info.columns)

# Junta a informação de quem é mandante/visitante na tabela de estatísticas
df_stats = pd.merge(
    df_stats,
    df_partida_info,
    left_on='partida_id', # Chave do df_stats
    right_on='ID',        # Chave do df_full
    how='left'
)

# Agora, identificamos a 'Posicao' (Mandante ou Visitante) de cada linha
df_stats['Posicao'] = 'Visitante' # Começamos assumindo que todos são visitantes
df_stats.loc[df_stats['clube'] == df_stats['HomeTeam'], 'Posicao'] = 'Mandante' # Corrigimos para quem é mandante


print("Tipos de dados originais em df_stats:")
print(df_stats.info())

# CÉLULA DE LIMPEZA - PODE SER RE-EXECUTADA COM SEGURANÇA

# 1. Começamos com uma cópia fresca dos dados brutos
df_stats = df_stats_raw.copy()

# Lista das colunas que são porcentagens
colunas_percentual = ['posse_de_bola', 'precisao_passes']

# Loop para limpar, converter e tratar nulos
for col in colunas_percentual:
    # Garante que a coluna seja tratada como texto
    df_stats[col] = df_stats[col].astype(str).str.replace('%', '', regex=False)
    
    # Converte para número de forma segura
    df_stats[col] = pd.to_numeric(df_stats[col], errors='coerce')
    
    # Divide por 100
    df_stats[col] = df_stats[col] / 100.0
    
    # Preenche nulos com a mediana
    mediana = df_stats[col].median()
    df_stats[col] = df_stats[col].fillna(mediana)

print("Conversão finalizada! Verificando o resultado com .describe():")
print(df_stats[['posse_de_bola', 'precisao_passes']].describe())

# Lista de colunas de valores com os nomes exatosm
valores_pivot = [
    'chutes', 'chutes_no_alvo', 'posse_de_bola', 'passes', 
    'precisao_passes', 'faltas', 'cartao_amarelo', 'cartao_vermelho', 
    'impedimentos', 'escanteios'
]

# Executando o pivot com os nomes de colunas corretos
df_stats_pivoted = df_stats.pivot_table(
    index='partida_id',
    columns='Posicao',  # CORRIGIDO: de 'posicao' para 'Posicao'
    values=valores_pivot
)

# Achatando os nomes das colunas
df_stats_pivoted.columns = [f'{stat}_{posicao}' for stat, posicao in df_stats_pivoted.columns]

print("\nTabela de estatísticas foi pivotada com sucesso!")
df_stats_pivoted.head()

In [3]:
# ==============================================================================
# BLOCO CONSOLIDADO FINAL (v3)
# ==============================================================================

# --- PASSO 0: COMEÇAR COM DADOS FRESCOS ---
df_stats = df_stats_raw.copy()
df_full = df_full_raw.copy()


# --- PASSO 1: LIMPEZA E CONVERSÃO (JÁ VALIDADO) ---
colunas_percentual = ['posse_de_bola', 'precisao_passes']
for col in colunas_percentual:
    df_stats[col] = df_stats[col].astype(str).str.replace('%', '', regex=False)
    df_stats[col] = pd.to_numeric(df_stats[col], errors='coerce')
    df_stats[col] = df_stats[col] / 100.0
    mediana = df_stats[col].median()
    df_stats[col] = df_stats[col].fillna(mediana)
print("Passo 1: Limpeza e conversão concluídos.")


# --- PASSO 2: CRIAR A COLUNA 'Posicao' ---
# CORRIGIDO AQUI: Usando os nomes de coluna corretos do df_full ('mandante', 'visitante')
df_partida_info = df_full[['ID', 'mandante', 'visitante']]

# Usando left_on e right_on para a junção com chaves de nomes diferentes
df_stats = pd.merge(
    df_stats,
    df_partida_info,
    left_on='partida_id',
    right_on='ID',
    how='left'
)

# CORRIGIDO AQUI: Usando o nome da coluna 'mandante' que veio do merge
df_stats['Posicao'] = 'Visitante'
df_stats.loc[df_stats['clube'] == df_stats['mandante'], 'Posicao'] = 'Mandante'
print("Passo 2: Coluna 'Posicao' criada com sucesso.")


# --- PASSO 3: EXECUTAR O PIVOT ---
valores_pivot = [
    'chutes', 'chutes_no_alvo', 'posse_de_bola', 'passes',
    'precisao_passes', 'faltas', 'cartao_amarelo', 'cartao_vermelho',
    'impedimentos', 'escanteios'
]

df_stats_pivoted = df_stats.pivot_table(
    index='partida_id',
    columns='Posicao',
    values=valores_pivot
)

df_stats_pivoted.columns = [f'{stat}_{posicao}' for stat, posicao in df_stats_pivoted.columns]

print("\nPasso 3: Tabela de estatísticas foi pivotada com sucesso!")
display(df_stats_pivoted.head())

Passo 1: Limpeza e conversão concluídos.
Passo 2: Coluna 'Posicao' criada com sucesso.

Passo 3: Tabela de estatísticas foi pivotada com sucesso!


,cartao_amarelo_Mandante,cartao_amarelo_Visitante,cartao_vermelho_Mandante,cartao_vermelho_Visitante,chutes_Mandante,chutes_Visitante,chutes_no_alvo_Mandante,chutes_no_alvo_Visitante,escanteios_Mandante,escanteios_Visitante,faltas_Mandante,faltas_Visitante,impedimentos_Mandante,impedimentos_Visitante,passes_Mandante,passes_Visitante,posse_de_bola_Mandante,posse_de_bola_Visitante,precisao_passes_Mandante,precisao_passes_Visitante
partida_id,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.81,0.81
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.81,0.81
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.81,0.81
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.81,0.81
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.81,0.81


In [4]:
# .tail(5) para ver dados mais recentes e ver se esta tudo certo
print("Amostra de 5 jogos recentes da tabela pivotada:")

display(df_stats_pivoted.tail(5))

Amostra de 5 jogos recentes da tabela pivotada:


,cartao_amarelo_Mandante,cartao_amarelo_Visitante,cartao_vermelho_Mandante,cartao_vermelho_Visitante,chutes_Mandante,chutes_Visitante,chutes_no_alvo_Mandante,chutes_no_alvo_Visitante,escanteios_Mandante,escanteios_Visitante,faltas_Mandante,faltas_Visitante,impedimentos_Mandante,impedimentos_Visitante,passes_Mandante,passes_Visitante,posse_de_bola_Mandante,posse_de_bola_Visitante,precisao_passes_Mandante,precisao_passes_Visitante
partida_id,,,,,,,,,,,,,,,,,,,,
8781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.81,0.81
8782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.81,0.81
8783,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.81,0.81
8784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.81,0.81
8785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.81,0.81


In [5]:
# --- TESTE DE VALIDAÇÃO PROFUNDA ---

# 1. Escolha um ID de partida recente para investigar
id_para_investigar = 8785

print(f"🕵️‍♂️ --- Investigando a Partida ID: {id_para_investigar} --- 🕵️‍♂️")
print("="*60)

# 2. Encontre a partida no arquivo de resultados original (para saber quem jogou)
print("\n[Passo 1] Informações do Jogo no arquivo 'df_full_raw':")
info_jogo_raw = df_full_raw.query(f"ID == {id_para_investigar}")
if not info_jogo_raw.empty:
    display(info_jogo_raw)
else:
    print("Jogo não encontrado em df_full_raw.")

print("="*60)

# 3. Encontre as estatísticas BRUTAS para essa partida no arquivo original
print("\n[Passo 2] Estatísticas BRUTAS no arquivo 'df_stats_raw':")
stats_jogo_raw = df_stats_raw.query(f"partida_id == {id_para_investigar}")
if not stats_jogo_raw.empty:
    display(stats_jogo_raw)
else:
    print(f"!!! ALERTA: Nenhuma estatística encontrada para a partida {id_para_investigar} no arquivo original !!!")

print("="*60)

# 4. Veja o resultado FINAL para essa mesma partida na nossa tabela processada
print(f"\n[Passo 3] Resultado FINAL em 'df_stats_pivoted' para a partida {id_para_investigar}:")
resultado_final = df_stats_pivoted.loc[[id_para_investigar]]
display(resultado_final)

🕵️‍♂️ --- Investigando a Partida ID: 8785 --- 🕵️‍♂️

[Passo 1] Informações do Jogo no arquivo 'df_full_raw':


,ID,rodata,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado
8784,8785,38,08/12/2024,16:05,Gremio,Corinthians,4-4-2,4-3-1-2,Renato Gaúcho,R. Díaz,Corinthians,Arena do Grêmio,0,3,RS,SP



[Passo 2] Estatísticas BRUTAS no arquivo 'df_stats_raw':


,partida_id,rodata,clube,chutes,chutes_no_alvo,posse_de_bola,passes,precisao_passes,faltas,cartao_amarelo,cartao_vermelho,impedimentos,escanteios
17568,8785,38,Gremio,0,0,NaN,0,NaN,0,0,0,0,0
17569,8785,38,Corinthians,0,0,NaN,0,NaN,0,0,0,0,0



[Passo 3] Resultado FINAL em 'df_stats_pivoted' para a partida 8785:


,cartao_amarelo_Mandante,cartao_amarelo_Visitante,cartao_vermelho_Mandante,cartao_vermelho_Visitante,chutes_Mandante,chutes_Visitante,chutes_no_alvo_Mandante,chutes_no_alvo_Visitante,escanteios_Mandante,escanteios_Visitante,faltas_Mandante,faltas_Visitante,impedimentos_Mandante,impedimentos_Visitante,passes_Mandante,passes_Visitante,posse_de_bola_Mandante,posse_de_bola_Visitante,precisao_passes_Mandante,precisao_passes_Visitante
partida_id,,,,,,,,,,,,,,,,,,,,
8785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.81,0.81
